# SwissPeaks Data

In this code, Swiss Peaks website is scraped. The raw data is then saved to a folder with race and year. No cleaning is done.


In [34]:
import requests
import json
import html
import pandas as pd
import time
import re
import unicodedata
import datetime
import random
import numpy as np
from datetime import timedelta

# Start the timer
start_time = time.time()

year = '2022'
url = 'https://my4.raceresult.com/184067/RRPublish/data/list?key=392b4c7157f2b40b4e5c3da0fb0c594d&listname=RESULTS%7COverall%20Results&page=results&contest=0&r=group&name=%231_Swiss%20Peaks%20360&f='


delay_seconds = 15

## Scraping Swiss Peaks 2022 Data from the website

In [35]:
# URL of the JSON file
print(url)

# Send a GET request to fetch the JSON data
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()

    # Optionally, save the data to a JSON file
    with open(f'Data/JSON/SwissPeaks360_{year}_DEM.json', 'w') as f:
        json.dump(data, f, indent=4)

    print(f"Data saved to 'SwissPeaks360_{year}_DEM.json'")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


https://my4.raceresult.com/184067/RRPublish/data/list?key=392b4c7157f2b40b4e5c3da0fb0c594d&listname=RESULTS%7COverall%20Results&page=results&contest=0&r=group&name=%231_Swiss%20Peaks%20360&f=
Data saved to 'SwissPeaks360_2022_DEM.json'


In [36]:

def convert_to_timedelta(time_str):
    """Convert time string from 'D:HH:MM:SS' to timedelta."""
    try:
        days, hours, minutes, seconds = map(int, time_str.split(':'))
        return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)
    except ValueError:
        return None  # Handle bad format gracefully

def reformat_name(name_str):
    parts = [p.strip() for p in name_str.split(',')]
    return f"{parts[1]} {parts[0]}" if len(parts) == 2 else name_str


with open(f'Data/JSON/SwissPeaks360_{year}_DEM.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)


    # Extract the specific race data
    race_participants = json_data['data']
#     print(race_participants)
    extracted_list = []

    for participant in race_participants:
#         print(participant)
        try:
            match = re.search(r'\[img:flags/([A-Z]{2})\.gif\]', participant[4])
            country_code = match.group(1) if match else ""
            
            
                    # Convert time
            raw_time = participant[8]
            td = convert_to_timedelta(raw_time)
            formatted_time = f"{raw_time.split(':')[0]} days {':'.join(raw_time.split(':')[1:])}"

                    # Format name
            formatted_name = reformat_name(participant[5])

            extracted_entry = {
                "Bib": participant[0],
                "ID": participant[1],
                "Status": participant[2],
                "Bib_1": participant[3],
                "Nationality": country_code,
                "Name": formatted_name,
                "Gender": participant[6],
                "Birth Year": participant[7],
                "Duration": formatted_time
            }
            extracted_list.append(extracted_entry)
        except:
            print(participant)
        

# # Display extracted data
# for item in extracted_list:
#     print(item)


[228]


In [37]:
SP_dem_df =  pd.DataFrame(extracted_list)   
SP_dem_df['Status'] = np.where(SP_dem_df['Status'] == 'DNF', 'DNF', 'Finished')
SP_dem_df['Year'] = year
SP_dem_df['Race'] = 'SP360'
SP_dem_df['PK'] = SP_dem_df['Race'] +'_' + SP_dem_df['Year'] +'_' + SP_dem_df['ID']    

SP_dem_df = SP_dem_df[[
     'ID', 'Name','Status', 'Nationality', 'Gender', 
    'Birth Year', 'Duration', 'Year', 'Race', 'PK',
'Bib','Bib_1']]

SP_dem_df.head()

,ID,Name,Status,Nationality,Gender,Birth Year,Duration,Year,Race,PK,Bib,Bib_1
0,1325,Galen Reynolds,Finished,CA,M,1984,3 days 13:51:26,2022,SP360,SP360_2022_1325,2,2
1,1589,Wataru Iino,Finished,JP,M,1979,3 days 14:38:46,2022,SP360,SP360_2022_1589,4,4
2,962,Javier Galve,Finished,ES,M,1982,3 days 14:56:01,2022,SP360,SP360_2022_962,30,30
3,979,Matteo Bolis,Finished,IT,M,1979,3 days 16:21:52,2022,SP360,SP360_2022_979,9,9
4,878,Philippe Debadier,Finished,FR,M,1978,3 days 21:47:58,2022,SP360,SP360_2022_878,28,28


In [39]:
### They dont match on 3 different rows
# WHat does that mean?
# No idea
SP_dem_df[SP_dem_df['Bib'] !=SP_dem_df['Bib_1']]  

,ID,Name,Status,Nationality,Gender,Birth Year,Duration,Year,Race,PK,Bib,Bib_1
41,867,Toño Figueroa,Finished,ES,M,1981,5 days 06:33:20,2022,SP360,SP360_2022_867,8022,802.2
113,934,Pierre Masset,Finished,BE,M,1994,6 days 06:08:37,2022,SP360,SP360_2022_934,8141,814.1
123,868,Benoit Vanacker,Finished,FR,M,1978,6 days 10:48:58,2022,SP360,SP360_2022_868,8042,804.2


In [40]:
SP_dem_df.to_excel(f'Data/SwissPeaks360_{year}_DEM.xlsx', index=False)

In [41]:
SP_dem_df[SP_dem_df['Status'] == 'DNF']

,ID,Name,Status,Nationality,Gender,Birth Year,Duration,Year,Race,PK,Bib,Bib_1
127,1061,Laura Lakio,DNF,FI,F,1976,days,2022,SP360,SP360_2022_1061,200,200
128,1534,Marta Poretti,DNF,IT,F,1976,days,2022,SP360,SP360_2022_1534,35,35
129,556,Olivier Beguin,DNF,BE,M,1975,days,2022,SP360,SP360_2022_556,129,129
130,161,Dominique Gauthier,DNF,FR,M,1958,days,2022,SP360,SP360_2022_161,165,165
131,1257,Osman Komurcu,DNF,GB,M,1973,days,2022,SP360,SP360_2022_1257,229,229
...,...,...,...,...,...,...,...,...,...,...,...,...
223,242,François Farquet,DNF,CH,M,1985,days,2022,SP360,SP360_2022_242,16,16
224,1060,Ramon Bermo,DNF,US,M,1967,days,2022,SP360,SP360_2022_1060,225,225
225,621,Julien Thiémard,DNF,CH,M,1982,days,2022,SP360,SP360_2022_621,77,77
226,834,François Mach,DNF,CH,M,1961,days,2022,SP360,SP360_2022_834,247,247


### Using bib column to extract station 

In [8]:
station_list = []
failed_to_fetch= []

n = 0
# Loop through the unique bibs in the DataFrame
for unique_id in list(SP_dem_df['ID'].unique()):  # Limit to just one bib for testing

    n = n+1
    print(n, ' - ', unique_id)
    
    # URL of the JSON file
    url = f'https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid={unique_id}'
    print(url)

    # Send a GET request to fetch the JSON data
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()
        stations = data['Splits']

        # Loop through the stations
        for station in stations:
            station_name = station['Name']
            exists = station['Exists']
            
            try:
                rolling_duration = station['Gun']
                td = convert_to_timedelta(rolling_duration)
                formatted_rolling_duration = f"{rolling_duration.split(':')[0]} days {':'.join(rolling_duration.split(':')[1:])}"

            
            except KeyError:  # Use KeyError since we're checking for a missing key
                formatted_rolling_duration = np.nan  # Assign NaN if the 'Sector' key is not found

                
            try:
                aid_station_duration = station['Sector']
                
                td = convert_to_timedelta(aid_station_duration)
                formatted_aid_station_duration = f"{aid_station_duration.split(':')[0]} days {':'.join(aid_station_duration.split(':')[1:])}"

            except KeyError:  # Use KeyError since we're checking for a missing key
                formatted_aid_station_duration = np.nan  # Assign NaN if the 'Sector' key is not found


            # Store the extracted info in the station list
            extracted_info = [unique_id, station_name, formatted_rolling_duration,exists, formatted_aid_station_duration]
            station_list.append(extracted_info)
            
    else:
        print(f"Failed to fetch data for {unique_id}. Status code: {response.status_code}")
        failed_to_fetch.append(unique_id)
        
    
    #### So we dont get blocked!!!!
    delay_seconds = random.randint(5,20)
    print('Delay before calling the server again: ', delay_seconds)
    print('*'*20)
    time.sleep(delay_seconds)



1325
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1325
Delay before calling the server again:  13
********************
1589
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1589
Delay before calling the server again:  11
********************
962
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=962
Delay before calling the server again:  7
********************
979
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=979
Delay before calling the server again:  19
********************
878
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=878
Delay before calling the server again:  13
********************
904
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=904
Delay before calling the server again:  19
****

Delay before calling the server again:  5
********************
965
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=965
Delay before calling the server again:  11
********************
986
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=986
Delay before calling the server again:  5
********************
690
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=690
Delay before calling the server again:  20
********************
895
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=895
Delay before calling the server again:  13
********************
586
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=586
Delay before calling the server again:  16
********************
626
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c5

Delay before calling the server again:  8
********************
893
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=893
Delay before calling the server again:  8
********************
660
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=660
Delay before calling the server again:  13
********************
891
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=891
Delay before calling the server again:  19
********************
1248
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1248
Delay before calling the server again:  13
********************
1009
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1009
Delay before calling the server again:  9
********************
175
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb

Delay before calling the server again:  6
********************
239
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=239
Delay before calling the server again:  18
********************
1098
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1098
Delay before calling the server again:  18
********************
1054
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1054
Delay before calling the server again:  12
********************
4
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=4
Delay before calling the server again:  20
********************
565
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=565
Delay before calling the server again:  5
********************
774
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c5

Delay before calling the server again:  5
********************
1134
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1134
Delay before calling the server again:  15
********************
9
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=9
Delay before calling the server again:  14
********************
734
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=734
Delay before calling the server again:  11
********************
1001
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=1001
Delay before calling the server again:  13
********************
596
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c594d&pid=596
Delay before calling the server again:  10
********************
270
https://my4.raceresult.com/184067/RRPublish/data/splits?key=392b4c7157f2b40b4e5c3da0fb0c

In [21]:
# Convert the station list to a DataFrame
aid_station_df = pd.DataFrame(station_list, columns=['ID', 'Aid Station',  'Aid Station Duration', 'Exists?','Rolling Duration'])
aid_station_df['Year'] = year
aid_station_df['Race'] = 'SP360'

aid_station_df['PK'] = aid_station_df['Race'] +'_' + aid_station_df['Year'] +'_' + aid_station_df['ID']    

# Preview the DataFrame
print(aid_station_df.head())
# aid_station_df.to_excel(f'Data/SwissPeaks360_{year}_aid_stations.xlsx', index=False)

    Bib      Aid Station Aid Station Duration  Exists? Rolling Duration  Year  \
0  1325  Départ Ulrichen      0 days 00:00:07     True              NaN  2022   
1  1325          Hiischi      0 days 00:04:35     True  0 days 00:04:29  2022   
2  1325          Mosbode      0 days 00:08:34     True  0 days 00:03:58  2022   
3  1325             Gere      0 days 00:12:35     True  0 days 00:04:01  2022   
4  1325       Geisshitte      0 days 00:18:32     True  0 days 00:05:57  2022   

    Race               PK  
0  SP360  SP360_2022_1325  
1  SP360  SP360_2022_1325  
2  SP360  SP360_2022_1325  
3  SP360  SP360_2022_1325  
4  SP360  SP360_2022_1325  


In [27]:
aid_station_df[aid_station_df['ID']== '1534']

,Bib,Aid Station,Aid Station Duration,Exists?,Rolling Duration,Year,Race,PK
7936,1534,Départ Ulrichen,0 days 00:00:12,True,NaN,2022,SP360,SP360_2022_1534
7937,1534,Hiischi,0 days 00:05:18,True,0 days 00:05:06,2022,SP360,SP360_2022_1534
7938,1534,Mosbode,0 days 00:10:05,True,0 days 00:04:48,2022,SP360,SP360_2022_1534
7939,1534,Gere,0 days 00:14:54,True,0 days 00:04:48,2022,SP360,SP360_2022_1534
7940,1534,Geisshitte,0 days 00:22:09,True,0 days 00:07:16,2022,SP360,SP360_2022_1534
...,...,...,...,...,...,...,...,...
7993,1534,Taney (R),4 days 16:00:40,True,0 days 04:30:00,2022,SP360,SP360_2022_1534
7994,1534,Lovenex,4 days 19:45:40,True,0 days 03:45:00,2022,SP360,SP360_2022_1534
7995,1534,Le Fréney (R),4 days 21:50:40,True,0 days 02:05:00,2022,SP360,SP360_2022_1534
7996,1534,Débarcadère,NaN,False,NaN,2022,SP360,SP360_2022_1534


In [26]:
aid_station_df.loc[aid_station_df['Exists?'] == False, 'Rolling Duration'] = np.nan
aid_station_df.loc[aid_station_df['Exists?'] == False, 'Aid Station Duration'] = np.nan

In [12]:
aid_station_df['Aid Station'].unique()

array(['Départ Ulrichen', 'Hiischi', 'Mosbode', 'Gere', 'Geisshitte',
       'Gonerlistafel', 'Üerlicherblase', 'Ulrichen (R)', 'Schossmatte',
       'Reckingen (R)', 'Chummefurgge', 'Chäserstatt (R)',
       'Fiesch (BV in)', 'Fiesh (BV out)', 'Furggerchäller',
       'Fleschbode (R)', 'Lengritz (R)', 'Giw (R)', 'Eisten (BV in)',
       'Eisten (BV out)', 'Grächen (R)', 'Jungu (R)', 'Augstbordpass',
       'Bluömatt (R)', 'Col de la Forcletta', 'Tsahèlett (R)',
       'Grimentz (BV in)', 'Grimentz (BV out)',
       'Cb. des Becs de Bosson (R)', 'Evolène (R)', 'Chemeuille (R)',
       'Grande Dixence (BV in)', 'Grande Dixence (BV out)',
       'Col de Prafleuri', 'Grand Désert', 'Louvie', 'Plampro (R)',
       'Cabane Brunet', 'Cb du Col de Mille (R)', 'Prassurny (R)',
       "Fênetre d'Arpette", 'Trient (R)', 'Finhaut (BV in)',
       'Finhaut (BV out)', 'Col de Fenestral', "Col d'Emaney",
       'Auberge de Salanfe (R)', 'Col de Susanfe', 'Barme (R)',
       'Chaux Palin (R)', "Porte

In [28]:
aid_station_df.to_excel(f'Data/SwissPeaks360_{year}_aid_stations.xlsx', index=False)

In [ ]:
# End the timer
end_time = time.time()

# Calculate and print execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.6f} seconds")